In [5]:
# Define paths to your dataset
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=20,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Randomly shear images
    zoom_range=0.2,  # Randomly zoom images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest')  # Fill in missing pixels

# Only rescaling for testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,  # Batch size
    class_mode='categorical')  # Multi-class labels

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,  # Batch size
    class_mode='categorical')  # Multi-class labels

# Load the VGG19 model, excluding the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional base
base_model.trainable = False

# Add new top layers for our specific task
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Use GlobalAveragePooling2D instead of Flatten
    Dense(512, activation='relu'),  # Fully connected layer with 512 neurons
    Dropout(0.5),  # Dropout layer for regularization
    Dense(4, activation='softmax')  # Output layer with 4 neurons (one for each class)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),  # Lower learning rate for fine-tuning
              loss='categorical_crossentropy',  # Loss function for multi-class classification
              metrics=['accuracy'])  # Metric to monitor

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,  # Number of epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size)

# Save the model
model.save('alzheimers_model_vgg19.h5')

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Visualize the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()



Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.
Epoch 1/30


C:\Users\T.Tejaswini\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  7/160 ━━━━━━━━━━━━━━━━━━━━ 5:44 2s/step - accuracy: 0.3438 - loss: 1.3550

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [2]:


# Define paths to your dataset
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=20,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Randomly shear images
    zoom_range=0.2,  # Randomly zoom images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest')  # Fill in missing pixels

# Only rescaling for testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,  # Batch size
    class_mode='categorical')  # Multi-class labels

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,  # Batch size
    class_mode='categorical')  # Multi-class labels

# Load the VGG19 model, excluding the top (fully connected) layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional base
base_model.trainable = False

# Add new top layers for our specific task
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Use GlobalAveragePooling2D instead of Flatten
    Dense(512, activation='relu'),  # Fully connected layer with 512 neurons
    Dropout(0.5),  # Dropout layer for regularization
    Dense(4, activation='softmax')  # Output layer with 4 neurons (one for each class)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),  # Lower learning rate for fine-tuning
              loss='categorical_crossentropy',  # Loss function for multi-class classification
              metrics=['accuracy'])  # Metric to monitor

# Number of epochs
epochs = 20

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,  # Number of epochs
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size)

# Save the model
model.save('alzheimers_model_vgg19.h5')

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Visualize the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()


Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.
Epoch 1/20


C:\Users\T.Tejaswini\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


160/160 ━━━━━━━━━━━━━━━━━━━━ 550s 3s/step - accuracy: 0.4077 - loss: 1.2142 - val_accuracy: 0.5016 - val_loss: 1.0149
Epoch 2/20
  1/160 ━━━━━━━━━━━━━━━━━━━━ 5:29 2s/step - accuracy: 0.5625 - loss: 0.9174

c:\Program Files\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


160/160 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.5625 - loss: 0.9174 - val_accuracy: 0.5806 - val_loss: 0.9360
Epoch 3/20
151/160 ━━━━━━━━━━━━━━━━━━━━ 32s 4s/step - accuracy: 0.4743 - loss: 1.0595

In [ ]:
# Function to predict the class of a single image
def predict_image(image_path, model, class_labels):
    # Load the image
    img = load_img(image_path, target_size=(150, 150))
    # Convert the image to array
    img_array = img_to_array(img)
    # Rescale the image
    img_array = img_array / 255.0
    # Expand dimensions to match the input shape of the model
    img_array = np.expand_dims(img_array, axis=0)
    
    # Predict the class
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    
    # Map the predicted class to the corresponding label
    predicted_label = class_labels[predicted_class[0]]
    return predicted_label

# Class labels as per the training data
class_labels = list(train_generator.class_indices.keys())

# Example usage
image_path = '26 (19).jpg'  # Replace with the path to your image
predicted_label = predict_image(image_path, model, class_labels)
print(f'The predicted label for the image is: {predicted_label}')
